# NRGV Data Analyst Interview Project

## Data Reconciliation

### Future Work - Ideas

1) Could test for count of underscores in file name to chack for additional encodings<br>
2) Using LAT/LONG and Date, determine sunset/sunrise; progamaticaly remove records when the sun is not out (from sunset to sunrise).<br>
2a) Eliminate these records during the injest stage -Document time-delta vs lines of code vs reduced records.<br>
3) Automate the detection of new state in dictionary.

### Notebook Objectives:

Data Reconciliation:
<ul>
    <li>Verify values</li>

-----------------------------

### Libraries

In [1]:
import requests as req     # DOWNLOADING URL VIA URL
import zipfile             # DEALING WITH ZIP COMPRESSED FILE
from io import BytesIO     # READING FILE FROM BUFFER
import os                  # DIRECTORY CREATION
import pandas as pd        # PANDAS DATAFRAME
import datetime            # DATE AND TIME
from datetime import datetime
import numpy as np

### Variables

In [2]:
# SWSD_SUMMARY_DATA DAYTIME 
sumry_daytime_path = 'C://Users//bruce//Documents//Data//SWSD_SUMMARY_DATA//ALL_STATES_SWSD_SUMMARY_DATA_DAYTIME.csv'

# SWSD_SUMMARY_DATA DAYTIME_AGGREGATIONS
sumry_daytime_agg_path = 'C://Users//bruce//Documents//Data//SWSD_SUMMARY_DATA//ALL_STATES_SWSD_SUMMARY_DATA_DAYTIME_AGGREGATIONS.csv'

states_dict = {
    "Alabama":"al",
    "Arizona":"az",
    "California":"ca",
    "Colorado":"co",
    "Florida":"fl",
    "Maine":"me",
    "New Mexico":"nm",
    "Texas":"tx"    
}

In [3]:
# DEFINE COLUMN NAMES
col_names = 'State','DataType','Latitude','Longitude','PVType','CapacityMW','Power(MW)_SUM','Power(MW)_MAX',\
'Power(MW)_MEAN','TimeIntervalMin'

# READ DATA
sumry_daytime = pd.read_csv(sumry_daytime_path, parse_dates=['LocalTime'])
sumry_daytime_agg = pd.read_csv(sumry_daytime_agg_path, header=0,names=col_names,skiprows=1)

-------------------------------------------------

### ALL_STATES_SWSD_SUMMARY_DATA_DAYTIME

In [4]:
sumry_daytime.head()

,LocalTime,Power(MW),State,DataType,Latitude,Longitude,PVType,CapacityMW,TimeIntervalMin
0,2006-01-01 04:35:00,0.0,AL,Actual,30.45,-88.25,UPV,70,5
1,2006-01-01 04:40:00,0.0,AL,Actual,30.45,-88.25,UPV,70,5
2,2006-01-01 04:45:00,0.0,AL,Actual,30.45,-88.25,UPV,70,5
3,2006-01-01 04:50:00,0.0,AL,Actual,30.45,-88.25,UPV,70,5
4,2006-01-01 04:55:00,0.0,AL,Actual,30.45,-88.25,UPV,70,5


In [5]:
sumry_daytime.dtypes

LocalTime          datetime64[ns]
Power(MW)                 float64
State                      object
DataType                   object
Latitude                  float64
Longitude                 float64
PVType                     object
CapacityMW                  int64
TimeIntervalMin             int64
dtype: object

In [6]:
# FILTERED FOR: STATE==AL

x = sumry_daytime[(sumry_daytime['State']=='AL')]['Power(MW)'].sum()

print("The Actual power produced in AL was: {:,.2f}".format(x))

The Actual power produced in AL was: 106,926,981.70


In [36]:
# FILTERED FOR: STATE==AL, DataType=='Actual'

x = sumry_daytime[(sumry_daytime['State']=='AL') & (sumry_daytime['DataType']=='Actual')]['Power(MW)'].sum()

print("The Actual power produced in AL was: {:,.2f}".format(x))

The Actual power produced in AL was: 91,487,339.80


In [38]:
# FILTERED FOR: STATE==AL, DataType=='Actual',  PVType=='UPV'

x = sumry_daytime[(sumry_daytime['State']=='AL') & (sumry_daytime['DataType']=='Actual')\
                  & (sumry_daytime['PVType']=='UPV')]['Power(MW)'].sum()

print("The Actual power produced in AL was: {:,.2f}".format(x))

The Actual power produced in AL was: 30,602,182.80


In [40]:
# FILTERED FOR: STATE==AL, Latitude between 31.87 and 34.4, Longitude between -102.51 and -86.7
# block["Age"].between(12, 15, inclusive = True) 

x = sumry_daytime[(sumry_daytime['State']=='AL') & (sumry_daytime['Latitude'].between(31.87,34.4, inclusive = True))\
                 & (sumry_daytime['Longitude'].between(-102.51,-86.7, inclusive = True))]['Power(MW)'].sum()

print("The Actual power produced in AL was: {:,.2f}".format(x))

The Actual power produced in AL was: 20,289,112.10


In [45]:
# FILTERED FOR: STATE==AL, Latitude between 31.87 and 34.4, Longitude between -102.51 and -86.7
# block["Age"].between(12, 15, inclusive = True) 

x = sumry_daytime[(sumry_daytime['State']=='AL') & (sumry_daytime['Latitude'] == 33.75)\
                 & (sumry_daytime['Longitude']== -86.75)]['Power(MW)'].sum()

print("The Actual power produced in AL was: {:,.2f}".format(x))

The Actual power produced in AL was: 732,960.30


In [13]:
# FILTERED FOR: DataType=='Actual'

x = sumry_daytime[(sumry_daytime['DataType']=='Actual')]['Power(MW)'].max()

print("The Max Power(MW) produced was: {:,.2f}".format(x))
#print(sumry_daytime[(sumry_daytime['DataType']=='Actual')] & (sumry_daytime['Power(MW)']==x))
sumry_daytime[sumry_daytime['Power(MW)']==x]

The Max Power(MW) produced was: 225.00


,LocalTime,Power(MW),State,DataType,Latitude,Longitude,PVType,CapacityMW,TimeIntervalMin
64291057,2006-03-08 12:10:00,225.0,FL,Actual,25.25,-81.15,UPV,252,5


------------------------------------------------------------------------------------

### ALL_STATES_SWSD_SUMMARY_DATA_DAYTIME_AGGREGATIONS 

In [41]:
sumry_daytime_agg.head()

,State,DataType,Latitude,Longitude,PVType,CapacityMW,Power(MW)_SUM,Power(MW)_MAX,Power(MW)_MEAN,TimeIntervalMin
0,AL,Actual,30.45,-88.25,UPV,70,1200879.0,61.0,17.688599,339450
1,AL,Actual,30.55,-88.25,DPV,38,630932.4,32.2,9.293451,339450
2,AL,Actual,30.55,-88.15,DPV,38,636177.5,31.9,9.370710,339450
3,AL,Actual,30.55,-87.75,DPV,36,597170.9,31.5,8.796154,339450
4,AL,Actual,30.55,-87.55,UPV,80,1363291.2,74.6,20.080884,339450


In [43]:
# FILTERED FOR: STATE==AL, Latitude between 31.87 and 34.4, Longitude between -102.51 and -86.7
# block["Age"].between(12, 15, inclusive = True) 

x = sumry_daytime_agg[(sumry_daytime_agg['State']=='AL') & \
                      (sumry_daytime_agg['Latitude'].between(31.87,34.4, inclusive = True))\
                      & (sumry_daytime_agg['Longitude'].between(-102.51,-86.7, inclusive = True))\
                      & ()]['Power(MW)_SUM'].sum()

print("The Actual power produced in AL was: {:,.2f}".format(x))

The Actual power produced in AL was: 20,289,112.10
